# MAI643 - Artificial Intelligence in Medicine

Project Assignment 1 - Spring Semester 2024

Student Name:    
Christina Ioanna Saroglaki   
Jianlin Ye 

UCY Email:     
saroglaki.christina-ioanna@ucy.ac.cy    
jye00001@ucy.ac.cy 

## Description 
----
This file contains the source code for the development of the cervical cancer prediction system.

**This does not contain the source code for the preliminary analysis.** The source code for the preliminary analysis is contained in the `pre-processing.ipynb` file.

### Import Libararies

In [ ]:
from IPython.display import clear_output

!pip install pyarrow imbalanced-learn

clear_output()

In [ ]:
import pandas as pd 
import numpy as np
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.set_printoptions(formatter={'float':"{:6.5g}".format})

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Overview

As per the authors, the chosen dataset focuses on indicators associated with the diagnosis of cervical cancer, encompassing various features such as demographic information, habits, and medical records​. In more detail, the data was gathered at "Hospital Universitario de Caracas" in Venezuela from a total of 858 patients​.

C. J. Fernandes Kelwin and J. Fernandes, “Cervical cancer (Risk Factors),” UCI Machine 
Learning Repository. 2017.

In [ ]:
risk_factor_df = pd.read_csv("risk_factors_cervical_cancer.csv", 
            na_values=["?"])

print("----------------------------------- Information -----------------------------------")
risk_factor_df.info()

# Data pre-processing steps
---
- Re-encoded missing values from "?" to `NaN`
- Handled missing and duplicate values
- Dropped zero variance features
- Removed outliers
- Split data into training, validation and test sets
- Handled target variable imbalance
- Performed dimensionality reduction

## Data cleaning
### Missing Values

First, we needed to find and manage the volume of missing values contained in the dataset.

In [ ]:
print("----------------------------------- Missing Values -----------------------------------")
missing_info = risk_factor_df.isnull().sum()
total_nan = missing_info.sum()
total_entries = risk_factor_df.size

# Print total NaN values
if (total_nan == 0):
    print("\nNo NaN values in the dataset.")
else:
    print("\nNaN values found in the dataset.")

    print("\nTotal NaN values in dataset: {}/{}".format(total_nan, total_entries))

    # Sort columns by the number of missing values
    nan_columns = missing_info.sort_values(ascending=False)

    print("\nTop 15 columns with missing values:\n")
    for i, (col, count) in enumerate(nan_columns.head(15).items(), 1):
        print("{:2}. {:35} : {:}".format(i, col, count))

We identified that the features `STDs: Time since first diagnosis` and `STDs: Time since last diagnosis` were filled with NaN values of about 92%. Because of the high percentage, it was impractical to either eliminate the affected observations or fill the missing values with the mean of columns. Consequently, these features were excluded from the dataset.

In [ ]:
risk_factor_df.drop(columns=["STDs: Time since first diagnosis", "STDs: Time since last diagnosis"], inplace=True)

To ensure the optimal performance of future models, we also set a **missing value threshold of 10 per row**. Any rows that exceeded this threshold were eliminated from the dataset because we determined they were missing significant information.

In [ ]:
# Rows containing NaN values
total_rows = len(risk_factor_df)
nan_rows = risk_factor_df.isna().any(axis=1).tolist().count(True)
print("\nTotal Rows containing NaN values in dataset: {}/{}".format(nan_rows, total_rows))

# Find rows that contain more than 10 NaN values
rows_to_del = risk_factor_df[risk_factor_df.isna().sum(axis=1) > 10].index

print("\nRows containing >10 NaN values: {}/{}".format(len(rows_to_del), total_rows))

# Remove rows
risk_factor_df.drop(rows_to_del, inplace=True)
risk_factor_df.reset_index(drop=True, inplace=True)

For the remaining columns, we managed the missing values depending on the column. In more detail, if the column contained binary values (0,1) then the row containing the missing value was deleted. Otherwise, the missing value was replaced with the mean of the column.

In [ ]:
print("--------------------------- Handling Missing Values ---------------------------")
print("----------------------------------- BEFORE -----------------------------------")
print("Number of rows before filling missing values: ", len(risk_factor_df))

# Display the number of missing values before filling
print("\nNumber of missing values per column before filling:")
print(risk_factor_df.isnull().sum())

# Fill missing values depending on the column
for col in risk_factor_df.columns:
    # If the column has more than 3 unique values, fill with mean of the column
    if risk_factor_df[col].nunique() > 3:
        risk_factor_df[col] = risk_factor_df[col].fillna(risk_factor_df[col].median())
    
# Drop rest NaN containing rows
risk_factor_df=risk_factor_df.dropna()
risk_factor_df.reset_index(drop=True, inplace=True)

In [ ]:
print("\n----------------------------------- AFTER -----------------------------------")
print("Number of rows after filling missing values: ", len(risk_factor_df))

# Display the number of missing values after filling
print("\nNumber of missing values per column after filling:")
print(risk_factor_df.isnull().sum())

### Duplicate Rows

Following the missing value analysis, we examined if the dataset contained any duplicate rows and removed them from the dataset.

In [ ]:
print("----------------------------------- Duplicate Rows -----------------------------------")
# Check for duplicate rows
duplicate_rows = risk_factor_df.duplicated()

# Count the number of duplicate rows
num_duplicates = duplicate_rows.sum()

if num_duplicates == 0:
    print("No duplicate rows found in the dataset.")
else:
    print(f"Found {num_duplicates} duplicate rows in the dataset.\n")

    # Display the duplicate rows indexes (if any)
    print("Duplicate rows indexes: {}\n".format(risk_factor_df[duplicate_rows].index.values))

    # Removing duplicate rows
    print("----------------------------- Removing Duplicates ----------------------------")
    print("----------------------------------- BEFORE -----------------------------------")
    print("Number of rows before removing duplicates: ", len(risk_factor_df))

    risk_factor_df.drop_duplicates(inplace=True)
    risk_factor_df.reset_index(drop=True, inplace=True)

    print("\n----------------------------------- AFTER -----------------------------------")
    print("Number of rows after removing duplicates: ", len(risk_factor_df))


### Remove zero variance features
MOving on, we also calculated the mean and standard deviation for each column. Columns with a standard deviation of 0 were omitted from the dataset because they did not add significant variability to the data since they contained the same value for all observations.

In [ ]:
mean_df = risk_factor_df.mean()
std_df = risk_factor_df.std()

# Print columns that have a standard deviation 0 (contain only one value)
print("Columns containing 1 value: {}\n".format(std_df[std_df==0].index.values))

risk_factor_df.drop(columns=["STDs:cervical condylomatosis", "STDs:AIDS"], inplace=True)

### Remove Outliers

The IQR (Inter Quartile Range) approach is the most commonly used and most trusted approach used in the research field to find outliers in a dataset. We utilised IQR to identify and remove outliers.

In [ ]:
# Function finding the unique values of each column in the dataframe
def find_unique_values_df(feat: pd.DataFrame):
    return {col: feat[col].unique() for col in feat}

def find_outliers(col, indices):
    obs = risk_factor_df[col].iloc[indices]
    unique_items, counts = np.unique(obs, return_counts=True)
    unique_items, counts = unique_items[::-1], counts[::-1]

    values_to_delete = unique_items[counts < 2 ]
    return values_to_delete

def delete_outliers(col, to_delete):
    if (to_delete.size != 0):
        rows_to_del = risk_factor_df.loc[risk_factor_df[col].isin(to_delete)].index.values.tolist()

        # Remove rows
        risk_factor_df.drop(rows_to_del, inplace=True)
        risk_factor_df.reset_index(drop=True, inplace=True)

In [ ]:
# Unique Values
unique_vals = find_unique_values_df(risk_factor_df)
# Identify non-binary columns
non_binary_cols = [col for col, vals in unique_vals.items() if len(vals) > 2]

for col in non_binary_cols:

    # IQR cannot be applied to columns with median 0
    if (risk_factor_df[col].median() != 0):
        Q3, Q1 = np.percentile(risk_factor_df[col], [75 ,25])
        IQR = Q3-Q1

        upper = Q3+(1.5*IQR)
        lower = Q1-(1.5*IQR)

        print(col)
        print("median: {}, upper fence: {}, lower fence: {}\n".format(risk_factor_df[col].median(), upper, lower))

        #Delete one occurrence observations outside the upper fence as outliers
        upper_to_delete = find_outliers(col, np.where(risk_factor_df[col] > upper)[0])
        delete_outliers(col, upper_to_delete)

        
        #Delete one occurrence observations outside the lower fence as outliers
        lower_to_delete = find_outliers(col, np.where(risk_factor_df[col] < lower)[0])
        delete_outliers(col, lower_to_delete)

In [ ]:
print("\nFinal dataset size: {} cols, {} rows".format(risk_factor_df.shape[1], risk_factor_df.shape[0]))

## Dataset splitting

### Target Variable

As previously stated, the final system will focus on predicting a single target variable. According to the literature, Pap tests primarily serve as preventative medical screenings, while the Schiller and colposcopy examinations are usually coupled with a biopsy to validate the results of the tests. Considering this literature and our prior analysis, we have selected the `Biopsy` feature as the target variable for our system. 

In [ ]:
X = risk_factor_df.drop(columns=["Hinselmann", "Schiller", "Citology", "Biopsy"])
y = risk_factor_df["Biopsy"]

X_train, X_temp, y_train, y_temp = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=42,
                                                    stratify = y, 
                                                    shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, 
                                                y_temp, 
                                                test_size=0.5,
                                                random_state=42,
                                                stratify = y_temp,
                                                shuffle=True)

## Handling Imbalanced Data
In order to address the imbalance within the target variable, we implemented the **SMOTE-ENN** method. Introduced by Batista et al. (2004), this method combines the ability of SMOTE to generate synthetic data points for the minority class, and ENN (Edited Nearest Neighbor), which eliminates data points from both classes that have a different class from the majority class among their K-Nearest Neighbors.

We opted for SMOTE-ENN over SMOTETomek due to its ability to remove noisy samples, thereby enhancing the quality of the dataset.


In [ ]:
smote_enn = SMOTEENN(random_state=42)

x_train_sm, y_train_sm = smote_enn.fit_resample(X_train, y_train)

In [ ]:
#Plot
color_1 = [px.colors.qualitative.Prism[0], px.colors.qualitative.Prism[1]]

y_train_counts = y_train.value_counts().reset_index()
y_train_counts.columns = ['Class', 'Count']
y_train_counts['Class'] = y_train_counts['Class'].map({0: 'Healthy', 1: 'Cervical Cancer'})

y_train_sm_counts = y_train_sm.value_counts().reset_index()
y_train_sm_counts.columns = ['Class', 'Count']
y_train_sm_counts['Class'] = y_train_sm_counts['Class'].map({0: 'Healthy', 1: 'Cervical Cancer'})

targ_fig = make_subplots(1, 2, specs=[[{"type":"domain"}, {"type":"domain"}]],
    subplot_titles=["Original Distribution", "Balanced Distribution"])

targ_fig.add_trace(go.Pie(labels=y_train_counts["Class"],
    values=y_train_counts["Count"],
    marker_colors=color_1), 1, 1)

targ_fig.add_trace(go.Pie(labels=y_train_sm_counts["Class"],
    values=y_train_sm_counts["Count"],
    marker_colors=color_1), 1, 2)

targ_fig.update_layout(title_text="Rows Containing NaN Values",
    width=850, height= 400,
    title_x=0.5)

targ_fig.show()

## Data Scaling and Encoding
Scaling and Encoding were only performed in the training set to avoid information leakage.

In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Specify the features to be scaled
features_to_scale = ["Smokes (packs/year)", "Hormonal Contraceptives (years)", "IUD (years)"]

# Scale the specified features and add new columns with suffix '_scaled'
x_train_sm[[f"{feature}_scaled" for feature in features_to_scale]] = scaler.fit_transform(x_train_sm[features_to_scale])

# Transform the scaled columns in the validation and test sets as well
X_val[[f"{feature}_scaled" for feature in features_to_scale]] = scaler.transform(X_val[features_to_scale])
X_test[[f"{feature}_scaled" for feature in features_to_scale]] = scaler.transform(X_test[features_to_scale])

#Drop original columns from the training set
x_train_sm.drop(columns=features_to_scale, inplace=True)
X_val.drop(columns=features_to_scale, inplace=True)
X_test.drop(columns=features_to_scale, inplace=True)


In [ ]:
# Define age categories, every 5 years as an intervals
age_intervals = [13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68, 73, 78, 83, 88]
labels = list(range(len(age_intervals) - 1))

# Encode the "Age" feature
x_train_sm['Age_encoded'] = pd.cut(x_train_sm['Age'], bins=age_intervals, labels=labels, right=False)
X_val['Age_encoded'] = pd.cut(X_val['Age'], bins=age_intervals, labels=labels, right=False)
X_test['Age_encoded'] = pd.cut(X_test['Age'], bins=age_intervals, labels=labels, right=False)

#Drop original column from the training set
x_train_sm.drop(columns=['Age'], inplace=True)
X_val.drop(columns=['Age'], inplace=True)
X_test.drop(columns=['Age'], inplace=True)

## Dimensionality Reduction - PCA

In [ ]:
# Scale data
minMaxScaler = MinMaxScaler()
x_train_rescaled = minMaxScaler.fit_transform(x_train_sm)
x_val_rescaled = minMaxScaler.transform(X_val)
x_test_rescaled = minMaxScaler.transform(X_test)

# 95% of variance
pca = PCA(n_components=11)
x_train_reduced = pca.fit_transform(x_train_rescaled)
x_val_reduced = pca.transform(x_val_rescaled)
x_test_reduced = pca.transform(x_test_rescaled)

# Model Development
---
For the prediction models we selected to utilize the models of Support Vector Machine (SVM) and the ensemble method of Random Forests.

# Model Evaluation
---